In [25]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import random_split
from torchvision import models
import pytorch_lightning as pl

import matplotlib.pyplot as plt

%matplotlib inline

import numpy as np
import os
import h5py

In [20]:
PATH_TO_TRAIN = "/deep/group/aihc-bootcamp-fall2021/lymphoma/processed/tma_patches/train.hdf5"
PATH_TO_VAL = "/deep/group/aihc-bootcamp-fall2021/lymphoma/processed/tma_patches/val.hdf5"
PATH_TO_TEST = "/deep/group/aihc-bootcamp-fall2021/lymphoma/processed/tma_patches/test.hdf5"

In [21]:
class HDF5Dataset(Dataset):
    
    def __init__(self, hdf5_path: str):
        
        self.hdf5_path = hdf5_path
        
        self.data = h5py.File(self.hdf5_path, "r")
        self.sizes = {k: self.data[k][()].shape[0] for k in self.data.keys()}
        
        
    def __len__(self):
        return sum(list(self.sizes.values()))
    
    def __getitem__(self, idx):
        
        patient_id = ""
        for k in self.sizes:
            if idx <= 0:
                patient_id = k
                break
            idx -= self.sizes[k]
        
        return self.data[patient_id][()][idx]

In [23]:
class SupervisedBaseline(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # init a pretrained resnet
        backbone = models.resnet50(pretrained=True)
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        self.feature_extractor = nn.Sequential(*layers)

        # use the pretrained model to classify cifar-10 (10 image classes)
        num_target_classes = 10
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)
        
        return x

In [24]:
# Datasets

train_dataset = HDF5Dataset(PATH_TO_TRAIN)
val_dataset = HDF5Dataset(PATH_TO_VAL)

train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)

NameError: name 'models' is not defined

In [27]:
# model
model = SupervisedBaseline()

# training
trainer = pl.Trainer(gpus=2, num_nodes=8, precision=16, limit_train_batches=0.5)
trainer.fit(model, train_loader, val_loader)

/sailhome/vrishk/miniconda3/envs/aihc/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:746: UserWarning: You requested multiple GPUs but did not specify a backend, e.g. `Trainer(accelerator="dp"|"ddp"|"ddp2")`. Setting `accelerator="ddp_spawn"` for you.
  rank_zero_warn(
Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


MisconfigurationException: No `training_step()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.